# House prices prediction (kaggle.com)
In the Ames Housing dataset on kaggle, we would like to predict house prices with regression models. Thanks to @Serigne on kaggle.com for some helpful hints. 

### Import Libraries

In [ ]:
import pandas as pd
from sklearn import model_selection, linear_model, ensemble, metrics
from sklearn.preprocessing import LabelEncoder
from matplotlib import pyplot as plt
%matplotlib inline
import numpy as np
import seaborn as sns
pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) #Limiting floats output to 3 decimal points
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
import xgboost as xgb
from sklearn.base import clone
from sklearn.kernel_ridge import KernelRidge

## 1. Import and explore the data 

In [ ]:
trData = pd.read_csv('train.csv')
testData = pd.read_csv('test.csv')
data_list = [trData, testData]

In [ ]:
print("The size of the training data is {}.".format(trData.shape))
print("The size of the test data is {}.".format(testData.shape))

The test and train data have almost the same number of samples (~1460)! 

In [ ]:
trData.head()

In [ ]:
trData.describe()

## 2. Clean, fill in the missing data, and remove outliers

What are the missing data? 

In [ ]:
trData.columns

In [ ]:
nullCounts = trData.isnull().sum()
nullCounts[nullCounts > 0]

In [ ]:
nullCountsTest = testData.isnull().sum()
nullCountsTest[nullCountsTest > 0]

In [ ]:
def CheckNull(df, feature):
    print(df[feature].isnull().sum())

### 2.1 Fill in the missing data
We fill in the missing data with either mean() or mode(). 

In [ ]:
testData['TotalBsmtSF'].fillna(testData['TotalBsmtSF'].mean(), inplace = True)
testData['GarageArea'].fillna(testData['GarageArea'].mean(), inplace = True)
testData['MSZoning'].fillna(testData['MSZoning'].mode()[0], inplace = True)
testData['SaleType'].fillna(testData['SaleType'].mode()[0], inplace = True)
for data in data_list:
    data['LotFrontage'].fillna(data['LotFrontage'].mean(), inplace = True)
    data['BsmtQual'].fillna(data['BsmtQual'].mode()[0], inplace = True)
    data['Functional'].fillna(data['Functional'].mode()[0], inplace = True)
    data['GarageYrBlt'].fillna(data['GarageYrBlt'].mean(), inplace = True)
    data['GarageCars'].fillna(data['GarageCars'].mode()[0], inplace = True)
    data['Electrical'].fillna(data['Electrical'].mode()[0], inplace = True)
    data['MasVnrType'].fillna(data['MasVnrType'].mode()[0], inplace = True)
    data['MasVnrArea'].fillna(data['MasVnrArea'].mean(), inplace = True)
    data['BsmtCond'].fillna(data['BsmtCond'].mode()[0], inplace = True)
    data['BsmtExposure'].fillna(data['BsmtExposure'].mode()[0], inplace = True)
    data['GarageType'].fillna(data['GarageType'].mode()[0], inplace = True)
    data['BsmtFinSF1'].fillna(data['BsmtFinSF1'].mean(), inplace = True)
    data['PoolQC'].fillna('None', inplace = True)
    data['MiscFeature'].fillna('None', inplace = True)
    data['Alley'].fillna('None', inplace = True)
    data['Fence'].fillna('None', inplace = True)

In [ ]:
for data in data_list:
    data['GarageTypeSmple'] = data['GarageType']
    data['GarageTypeSmple'] = data['GarageType'].replace(['Basment', 'CarPort', '2Types'], 'Rare')
    data['ExterCondSmple'] = data['ExterCond'].replace(['Fa', 'Ex', 'Po'], 'Rare')
    data['HasDeckPorch'] = ((data['WoodDeckSF'] > 0) | (data['OpenPorchSF'] > 0) | (data['EnclosedPorch']) > 0 |
                            (data['3SsnPorch'] > 0) | (data['ScreenPorch'] > 0)) * 1
    data['HasPool'] = (data['PoolArea'] > 0) * 1
    data['HalfFullBath'] = data['FullBath'] + data['HalfBath']
    data['SaleTypeSmple'] = data['SaleType'].replace(['CoD', 'ConLD', 'ConLw', 'ConLI', 'Oth', 'Con', 'CWD'], 'Rare')

### 2.2 Convert to numeric
We now convert the non-numeric values to numeric values. 

Transform categorial type features to numeric:

In [ ]:
combinedData = pd.concat(objs = [trData, testData], axis = 0).reset_index(drop = True)
Ntr = len(trData)
Ntest = len(testData)
Ntot = len(combinedData)
label = LabelEncoder()
label.fit(combinedData['Neighborhood']) 
labelZone = LabelEncoder()
labelZone.fit(combinedData['MSZoning'])
labelBldgType = LabelEncoder()
labelBldgType.fit(combinedData['BldgType'])
labelHouseStyle = LabelEncoder()
labelHouseStyle.fit(combinedData['HouseStyle'])
labelListOld = ['Neighborhood', 'MSZoning', 'BldgType', 'HouseStyle', 'Foundation', 'CentralAir', 
                'PavedDrive', 'SaleType', 'SaleCondition', 'BsmtQual', 'Heating', 'Functional', 'Street', 
               'ExterCond', 'RoofMatl', 'RoofStyle', 'LandContour', 'Condition1', 'Condition2', 'Electrical',
               'MasVnrType', 'ExterQual', 'GarageTypeSmple', 'ExterCondSmple', 'SaleTypeSmple', 'BsmtCond', 'PoolQC', 
               'MiscFeature', 'Alley', 'Fence']
labelListNew = ['NeighborhoodCode', 'MSZoningCode', 'BldgTypeCode', 'HouseStyleCode', 'FoundationCode', 'CentralAirCode', 
                'PavedDriveCode', 'SaleTypeCode', 'SaleConditionCode', 'BsmtQualCode', 'HeatingCode', 'FunctionalCode', 
               'StreetCode', 'ExterCondCode', 'RoofMatlCode', 'RoofStyleCode', 'LandContour', 'Condition1Code', 
               'Condition2Code', 'ElectricalCode', 'MasVnrTypeCode', 'ExterQualCode', 'GarageTypeSmpleCode', 'ExterCondSmpleCode',
               'SaleTypeSmpleCode', 'BsmtCondCode', 'PoolQCCode', 'MiscFeatureCode', 'AlleyCode', 'FenceCode']

for data in data_list: 
    for i in range(len(labelListOld)):
        label_encoder = LabelEncoder()
        label_encoder.fit(combinedData[labelListOld[i]])
        data[labelListNew[i]] = label_encoder.transform(data[labelListOld[i]])

### Graph the price (output) and features (input) distributions:

In [ ]:
f, ax = plt.subplots(1, 2, figsize = (15, 5))
FZ = 15
ax[0].hist(trData['SalePrice'])
ax[0].set_title('House prices distribusion (USD)', fontsize = FZ)
ax[0].set_xlabel('Sale Price', fontsize = FZ)
ax[0].set_ylabel('Counts', fontsize = FZ)

ax[1].hist(trData['SalePrice'] / trData['GrLivArea'])
ax[1].set_title('House prices per Living Area distribusion (USD)', fontsize = FZ)
ax[1].set_xlabel('Sale Price', fontsize = FZ)
ax[1].set_ylabel('Counts', fontsize = FZ)

In [ ]:
trData['PricePerArea'] = trData['SalePrice'] / trData['GrLivArea']
trData['PricePerArea'].mean()

For simplicity we will convert the price to $1000:

In [ ]:
trData['SalePriceK'] = trData['SalePrice'] / 1000

The average house price per square feet of living area (GrLivArea) is around 120 USD. 

In [ ]:
trData["SalePriceLog"] = np.log1p(trData["SalePrice"])

### 2.3 Remove the outliers

In [ ]:
plt.scatter(x = trData['GrLivArea'], y = trData['SalePrice'])
plt.grid()
plt.ylabel('SalePrice', fontsize=13)
plt.xlabel('GrLivArea', fontsize=13)

In [ ]:
trData.drop(trData[(trData['GrLivArea']>4000) & (trData['SalePrice']<300000)].index, inplace = True)
combinedData = pd.concat(objs = [trData, testData], axis = 0).reset_index(drop = True)

After removing the two outliers:

In [ ]:
sns.regplot(x = trData['GrLivArea'], y = trData['SalePrice'] /trData['GrLivArea'])
plt.grid()
plt.ylabel('SalePrice', fontsize=13)
plt.xlabel('GrLivArea', fontsize=13)

Another possiblity for outliers is the price per area. There is only one house with a price per square foot less than $31, and the next lowest price is $40, so, we remove the cheapest one:

In [ ]:
trData[trData['SalePrice'] / trData['GrLivArea'] < 31]

In [ ]:
trData.drop(trData[trData['SalePrice'] / trData['GrLivArea'] < 31].index, inplace = True)

In [ ]:
plt.scatter(x = trData['GrLivArea'], y = trData['SalePrice'] / trData['GrLivArea'])

In [ ]:
sns.regplot(x = trData['OverallQual'], y = trData['SalePrice'] / trData['GrLivArea'])
plt.grid()

In [ ]:
trData['PoolQC'].value_counts()

## 3. Feature engineering
We investigate the correlation between the features and the sale price.

In [ ]:
f, ax = plt.subplots(3, 3, figsize = (12, 10))
plt.subplot(3, 3, 1)
sns.regplot(x = 'GrLivArea', y = 'SalePriceK', data = trData)
plt.subplot(3, 3, 2)
sns.barplot(x = 'BedroomAbvGr', y = 'SalePriceK', data = trData)
plt.subplot(3, 3, 3)
sns.regplot(x = 'YearBuilt', y = 'SalePriceK', data = trData)
plt.subplot(3, 3, 4)
sns.barplot(x = 'OverallCond', y = 'SalePriceK', data = trData)
plt.subplot(3, 3, 5)
sns.barplot(x = 'OverallQual', y = 'SalePriceK', data = trData)
plt.subplot(3, 3, 6)
sns.barplot(x = 'MoSold', y = 'SalePriceK', data = trData)
plt.subplot(3, 3, 7)
sns.barplot(x = 'NeighborhoodCode', y = 'SalePriceK', data = trData)
plt.subplot(3, 3, 8)
sns.barplot(x = 'MSZoningCode', y = 'SalePriceK', data = trData)
plt.subplot(3, 3, 9)
sns.barplot(x = 'Condition2', y = 'SalePriceK', data = trData)
plt.tight_layout() 

In [ ]:
# tmpData = trData
# tmpData['1stFlrSF_2ndFlrSF'] = trData['1stFlrSF'] + trData['2ndFlrSF']
sns.barplot(x = 'Alley', y = 'SalePriceK', data = trData)
print(trData['Alley'].value_counts())
CheckNull(combinedData, 'BsmtCond')

In [ ]:
CheckNull(combinedData, 'ExterQual')

In [ ]:
for data in data_list:
    data['GarageCarsSmple'] = data['GarageCars'].replace(4, 3)

In [ ]:
sns.barplot(x = 'HalfFullBath', y = 'SalePriceK', data = trData)
print(trData['HalfFullBath'].value_counts())

In [ ]:
sns.barplot(x = 'HasDeckPorch', y = 'SalePriceK', data = trData)
print(trData['HasDeckPorch'].value_counts())

In [ ]:
sns.barplot(x = 'FireplaceQu', y = 'SalePriceK', data = trData)
print(trData['FireplaceQu'].value_counts())
CheckNull(combinedData, 'FireplaceQu')

Since only 'Gd' condition has a significant different with the rest, we define a new feature: 

In [ ]:
for data in data_list:
    data['BsmtExposureGd'] = (data['BsmtExposure'] == 'Gd') * 1

In [ ]:
for data in data_list:
    data['TotalSF'] = data['TotalBsmtSF'] + data['1stFlrSF'] + data['2ndFlrSF']

In [ ]:
f, ax = plt.subplots(3, 3, figsize = (12, 10))
plt.subplot(4, 3, 1)
sns.barplot(x = 'HouseStyleCode', y = 'SalePriceK', data = trData)
plt.subplot(4, 3, 2)
sns.barplot(x = 'ExterCond', y = 'SalePriceK', data = trData)
plt.subplot(4, 3, 3)
sns.barplot(x = 'CentralAir', y = 'SalePriceK', data = trData)
plt.subplot(4, 3, 4)
sns.barplot(x = 'PavedDrive', y = 'SalePriceK', data = trData)
plt.subplot(4, 3, 5)
sns.barplot(x = 'LotConfig', y = 'SalePriceK', data = trData)
plt.subplot(4, 3, 6)
sns.barplot(x = 'BsmtQual', y = 'SalePriceK', data = trData)
plt.subplot(4, 3, 7)
sns.barplot(x = 'NeighborhoodCode', y = 'SalePriceK', data = trData)
plt.subplot(4, 3, 8)
sns.barplot(x = 'Functional', y = 'SalePriceK', data = trData)
plt.subplot(4, 3, 9)
sns.regplot(x = 'YearRemodAdd', y = 'SalePriceK', data = trData)
plt.tight_layout() 
plt.subplot(4, 3, 10) 
sns.barplot(x = 'BsmtQual', y = 'SalePriceK', data = trData)
plt.subplot(4, 3, 11) 
sns.barplot(x = 'OverallCond', y = 'SalePriceK', data = trData) 
plt.subplot(4, 3, 12) 
sns.regplot(x = 'GarageYrBlt', y = 'SalePriceK', data = trData)

House age seems to be an important feature correlated with the saleprice:

In [ ]:
trData['YearBuilt'].describe()

In [ ]:
for data in data_list:
    data['HouseAge'] = data['YrSold'] - data['YearBuilt']
    data['YrfromRemod'] = data['YrSold'] - data['YearRemodAdd']
    data['YrSoldFrom2006'] = data['YrSold'] - 2006
    data['YrBuiltFrom1971'] = data['YearBuilt'] - 1971

In [ ]:
sns.regplot(x = 'YearBuilt', y = 'PricePerArea', data = trData)

In [ ]:
cor_col = ['GrLivArea', 'HouseAge', 'OverallCond', 'OverallQual', 'KitchenAbvGr', 'GarageArea', 
           'Functional', 'TotalBsmtSF',  'MSSubClass', 'NeighborhoodCode', 'MSZoningCode',
           'BldgTypeCode', 'HouseStyleCode', 'FoundationCode', 'CentralAirCode', 'PavedDriveCode', 
           'LotArea', 'RoofStyleCode', 'SalePrice']

In [ ]:
def correlation_heatmap(df):
    _ , ax = plt.subplots(figsize =(14, 12))
    colormap = sns.diverging_palette(220, 10, as_cmap = True)
    
    _ = sns.heatmap(
        df.corr(), 
        cmap = colormap,
        square=True, 
        cbar_kws={'shrink':.9 }, 
        ax=ax,
        annot=True, 
        linewidths=0.1,vmax=1.0, linecolor='white',
        annot_kws={'fontsize':12 }
    )
    
    plt.title('Pearson Correlation of Features', y=1.05, size=15)

correlation_heatmap(trData[cor_col])

In [ ]:
combinedData['Condition2'].value_counts()

## 4. Train the models and predict

In [ ]:
selected_features = ['GrLivArea', 'HouseAge', 'OverallQual', 'BedroomAbvGr', 'KitchenAbvGr', 
                     'FullBath', 'GarageArea', 'Fireplaces', 'MSSubClass', 
                     'MSZoningCode', 'BldgTypeCode', 'HouseStyleCode', 
                     'FoundationCode', 'CentralAirCode', 'PavedDriveCode', 'LotArea', 
                     'YearRemodAdd', 'BsmtQualCode', 'TotRmsAbvGrd', 'OverallCond', 'LandContour',
                     'MasVnrTypeCode', 'BsmtExposureGd', 'YearBuilt', 'GarageTypeSmpleCode', 'GarageCarsSmple',
                     'HasDeckPorch', 'BsmtFinSF1', 'NeighborhoodCode', 'TotalSF']
# good GarageCarsSmple HouseStyleCode CentralAirCode BsmtQualCode OverallCond Fireplaces TotalBsmtSF MSSubClass MSZoningCode
# bad BsmtExposureGd GarageArea FoundationCode LandContour HasDeckPorch FullBath MasVnrTypeCode BedroomAbvGr
# bad PavedDriveCode BldgTypeBsmtExposureGdCode GarageTypeSmpleCode KitchenAbvGr
selected_features = ['GrLivArea', 'OverallQual', 'BsmtFinSF1', 'HouseAge', 'NeighborhoodCode', 'YearRemodAdd', 
                    'GarageCarsSmple', 'LotArea', 'OverallCond', 'Fireplaces',
                    'HouseStyleCode', 'MSSubClass',  'BsmtQualCode', 'PavedDriveCode', 'MasVnrTypeCode', 'TotalSF'
                    ]

X = trData[selected_features]
y = trData['SalePrice']

In [ ]:
Xtrain, Xtest, ytrain, ytest = model_selection.train_test_split(X, y, test_size = 0.5, train_size = 0.5, random_state=1)

In [ ]:
n_folds = 10
def rmsle_cv(model):
    kf = model_selection.KFold(n_folds, shuffle=True, random_state=42).get_n_splits(X)
    rmse= np.sqrt(-model_selection.cross_val_score(model, X, y, scoring="neg_mean_squared_error", cv = kf))
    return(sum(rmse)/n_folds)

In [ ]:
clfList = [linear_model.LinearRegression(), ensemble.RandomForestRegressor(), ensemble.GradientBoostingRegressor(),
          xgb.XGBRegressor()]
cvSplit = model_selection.ShuffleSplit(n_splits = 10, train_size = 0.5, test_size = 0.5, random_state = 0)
maxDepthList = [2, 4, 6, 8, 10]
nEstimatorsList = [10, 50, 100, 300]
etaList = [0.1, 0.05, 0.01]
gridSeedList = [0]
gammaList = [0, 0.1, 0.2, 0.5, 1]
colsample_bytreeList = [0.4, 0.5, 0.6, 1]
gridBool = [True, False]
paramGridList = [
                [{'fit_intercept': gridBool}], [{'max_depth': maxDepthList, 'random_state': gridSeedList}],
                [{'n_estimators': nEstimatorsList, 'max_depth': maxDepthList, 'random_state': gridSeedList}], 
                [{'max_depth': maxDepthList, 'gamma': gammaList, 'colsample_bytree': colsample_bytreeList}],
                [{}]
                ]
bestScoreList = []
for clf, param in zip(clfList, paramGridList):
    bestSearch = model_selection.GridSearchCV(estimator = clf, param_grid = param, 
                                              cv = cvSplit, scoring = 'neg_mean_squared_error', n_jobs = 4)
    bestSearch.fit(X, y)
    bestParam = bestSearch.best_params_
    bestScore = round((-bestSearch.best_score_)**0.5, 5) / 1000
    print('The best parameter for {} is {} with a runtime of seconds with an error of {}'.format(clf.__class__.__name__, bestParam, bestScore))
    clf.set_params(**bestParam) 
    bestScoreList.append(bestScore)
print("--"*45, "\nMax cross-validation score is {}".format(round(min(bestScoreList), 5)))
print("--"*45, "\nAverage cross-validation score is {}".format(sum(sorted(bestScoreList, reverse=False)[0:3]) / 3))

In [ ]:
# Thanks to Serigne on kaggle.com
class AveragingModels():
    def __init__(self, models):
        self.models = models
        
    # we define clones of the original models to fit the data in
    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]
        
        # Train cloned base models
        for model in self.models_:
            model.fit(X, y)

        return self
    
    #Now we do the predictions for cloned models and average them
    def predict(self, X):
        predictions = np.column_stack([
            model.predict(X) for model in self.models_
        ])
        return np.mean(predictions, axis=1)

In [ ]:
LassoRobust = make_pipeline(RobustScaler(), linear_model.Lasso(alpha =100, random_state=1))
ENet = make_pipeline(RobustScaler(), linear_model.ElasticNet(alpha=0.5, l1_ratio=.9, random_state=3))
KRR = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)

In [ ]:
averagingC = AveragingModels(models = (clfList[2], clfList[3]))
averagingC.fit(Xtrain, ytrain) # Note we fit the Whole X, y
arpredict = averagingC.predict(Xtest)
print(metrics.mean_squared_log_error(ytest, arpredict)**0.5)
predData = pd.DataFrame({'Index':ytest.index, 'SalePrice': ytest.values, 'SalePricePredicted':arpredict,
                         'Error': arpredict - ytest.values})

In [ ]:
averagingC.fit(X, y) # Note we fit the Whole X, y
arpredict = averagingC.predict(Xtest)
print(metrics.mean_squared_log_error(ytest, arpredict)**0.5)
predData = pd.DataFrame({'Index':ytest.index, 'SalePrice': ytest.values, 'SalePricePredicted':arpredict,
                         'Error': arpredict - ytest.values})

In [ ]:
# votingC = clfList[2]
# votingC.fit(Xtrain, ytrain) # Note we fit the Whole X, y
# arpredict = votingC.predict(Xtest)
# print(metrics.mean_squared_log_error(ytest, arpredict)**0.5)
# predData = pd.DataFrame({'Index':ytest.index, 'SalePrice': ytest.values, 'SalePricePredicted':arpredict,
#                          'Error': arpredict - ytest.values})

In [ ]:
# votingC = clfList[2]
# votingC.fit(X, y) # Note we fit the Whole X, y
# arpredict = votingC.predict(Xtest)
# print(metrics.mean_squared_log_error(ytest, arpredict)**0.5)
# predData = pd.DataFrame({'Index':ytest.index, 'SalePrice': ytest.values, 'SalePricePredicted':arpredict,
#                          'Error': arpredict - ytest.values})

In [ ]:
trsh = 100000
print(len(Xtest[abs(arpredict - ytest.values) > trsh]))
predData[abs(arpredict - ytest.values) > trsh]

In [ ]:
plt.hist(Xtest[abs(arpredict - ytest.values) > trsh]['OverallQual'])

In [ ]:
ytest[abs(arpredict - ytest.values) > 100000]

In [ ]:
testDataTemp = testData[selected_features]
arpredict = averagingC.predict(testDataTemp)
ypredict = pd.DataFrame({'Id': testData['Id'], 'SalePrice':arpredict})
ypredict.to_csv('../predictions.csv', index = False)

In [ ]:
ypredict.head()